In [1]:
import os
import json
import pandas as pd

In [2]:
file_path = '../arXivEdits/data/edits/edits_processed.csv'
# Step 1: Verify the file path
if not os.path.exists(file_path):
    print("File not found:", file_path)
else:
    print("File found:", file_path)

df = pd.read_csv(file_path)

# Drop a column (e.g., 'column2')
df = df.drop('Unnamed: 0', axis=1)
df

File found: ../arXivEdits/data/edits/edits_processed.csv


,before,after,label
0,"In particular , modern imaging and machine lea...","In particular , modern imaging and machine lea...",Lang-professional-Improve-style
1,"According to the proof of Proposition 2 , [MAT...","According to the proof of Corollary 1 , [MATH]...",Lang-accurate-spefific
2,"However , depending somewhat on the Hubble con...","However , depending somewhat on the Hubble con...",Format
3,"However , depending somewhat on the Hubble con...","However , depending somewhat on the Hubble con...",Lang-accurate-spefific
4,This indicates that the mass fraction of in-si...,This indicates that the mass fraction of in-si...,Lang-accurate-spefific
...,...,...,...
2117,We found in Ref. [CITATION] that various patte...,We found in Ref. [CITATION] that various patte...,Lang-accurate-spefific
2118,We found in Ref. [CITATION] that various patte...,We found in Ref. [CITATION] that various patte...,Content
2119,We found in Ref. [CITATION] that various patte...,We found in Ref. [CITATION] that various patte...,Content
2120,We found in Ref. [CITATION] that various patte...,We found in Ref. [CITATION] that various patte...,Lang-professional-Improve-style


In [3]:
repo="arXivEdits"
n_sampels_pilot_study=15
n_sampels_pilot_study_list=[5, 5, 5]
n_samples_comparing_group=165
random_seed=41
group_by=['label']

# repo="arXivEdits"
# n_sampels_pilot_study=15
# n_sampels_pilot_study_list=[5, 5, 5]
# n_samples_comparing_group=165
# random_seed=41
# group_by=['label']

In [4]:
# Define the sampling function
def sample_data(group, n=1, random_state=42):
    return group.sample(n=min(len(group), n), random_state=random_state)

# Sample n rows per 'domain' and 'label' combination with a random seed
sampled_df = df.groupby(group_by).apply(sample_data, n=n_sampels_pilot_study, random_state=random_seed).reset_index(drop=True)

# Display the sampled DataFrame
sampled_df

,before,after,label
0,A step into this direction was initiated by Ha...,A step into this direction was initiated by Ha...,Content
1,In this process step the unitary constraints a...,via a passive approach .,Content
2,It follows that the approximate expressions [R...,It follows that the approximate expressions [R...,Content
3,Hence we conclude that the wave functions we u...,Hence we conclude that the wave functions we u...,Content
4,"To be slightly more precise , we recall that t...","To be slightly more precise , we recall that t...",Content
...,...,...,...
100,"Finally , this phenomenon of stability enhance...","Finally , this phenomenon of stability enhance...",Lang-professional-Improve-style
101,"Cost-optimisation technology models , in norma...","Cost-optimisation technology models , in norma...",Lang-professional-Improve-style
102,It converges rapidly in the weak and strong co...,This leads to a rapid convergence in the weak...,Lang-professional-Improve-style
103,The model describing the main factors that aff...,The model describing the main factors that aff...,Lang-professional-Improve-style


In [5]:
# Get the indices of the sampled data
sampled_indices = sampled_df.index

# Drop the sampled data to get the remaining data
remaining_df = df.drop(sampled_indices)

remaining_df = remaining_df.reset_index(drop=True)

In [6]:
# Convert DataFrame to JSON Lines format and save to a file
sampled_df.to_json(f"./{repo}/pilot_study_{repo.lower()}.json", orient='records', lines=True)

In [7]:

for idx, n_samples in enumerate(n_sampels_pilot_study_list):
    # Sample n rows per 'domain' and 'label' combination with a random seed
    sampled_iter_df = sampled_df.groupby(group_by).apply(sample_data, n=n_samples, random_state=random_seed).reset_index(drop=True)

    sampled_iter_df.to_json(f"./{repo}/pilot_study_{repo.lower()}_iter{idx+1}.json", orient='records', lines=True)

    # Get the indices of the sampled data
    sampled_iter_indices = sampled_iter_df.index

    # Drop the sampled data to get the remaining data
    sampled_df = sampled_df.drop(sampled_iter_indices)
    # Reset the index
    sampled_df = sampled_df.reset_index(drop=True)

In [8]:
## dataset for comparing group

# Sample n rows per 'domain' and 'label' combination with a random seed
comparing_df = remaining_df.groupby(group_by).apply(sample_data, n=n_samples_comparing_group, random_state=random_seed).reset_index(drop=True)
comparing_df.to_json(f"./{repo}/comparing_group_{repo.lower()}.json", orient='records', lines=True)